# Importing

In [14]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import poisson

# **Initialization**
## Maximum
 * MAX_Capacity = 20
 * MAX_transfer = 5

## FIRST of the appointment
 * A_REQUEST_FIRST = 3
 * B_REQUEST_FIRST  = 4

## END of the appointment
 * A_REQUEST_END = 3
 * B_REQUEST_END = 2

## gamma factor(default 0.9)
 * DISCOUNT = 0.9

## Define reward
 * CREDIT = 10
 * COST = 2

# **Define Poisson disturbution**

In [15]:
# Probability for poisson distribution
poisson_dict = dict()
def poisson_probability(n, mu):
    global poisson_dict
    key = n * 10 + mu
    if key not in poisson_dict:
        poisson_dict[key] = poisson.pmf(n, mu)
    return poisson_dict[key]

# **Return states**
## Overall info
 * state: [# of capacity in first company , # of capacity in second company]
 * action: 
    * positive if transferin from first company to second one,
    * negative if transfering from second company to frist one,
    
* stateValue: state value matrix



## Creat a function

In [19]:
def return_value(state, action , state_value ,MAX_Capacity, MAX_transfer, A_REQUEST_FIRST, B_REQUEST_FIRST,
                 A_REQUEST_END ,B_REQUEST_END ,DISCOUNT ,CREDIT,COST ,UPPER_BOUND):                                                   

    # initailize total return
    value_func = 0.0

    # cost for moving cars
    value_func -= COST * abs(action)

    # moving cars
    CAPACITY_A = min(state[0] - action, MAX_Capacity)
    CAPACITY_B = min(state[1] + action, MAX_Capacity)

    # go through all possible rental requests
    for capacity_request_A in range(UPPER_BOUND):
        for capacity_request_B in range(UPPER_BOUND):
            # probability for current combination of companies requests
            prob = poisson_probability(capacity_request_A, A_REQUEST_FIRST) * \
                poisson_probability(capacity_request_B,B_REQUEST_FIRST)

            capacity_a = CAPACITY_A
            capacity_b = CAPACITY_B

            # valid companies requests should be less than actual # of capacities
            valid_capacity_a = min(capacity_a, capacity_request_A)
            valid_capacity_b = min(capacity_b ,capacity_request_B)

            # get credits for transfering
            reward = (valid_capacity_a + valid_capacity_b) * CREDIT
            capacity_a -= valid_capacity_a
            capacity_b -= valid_capacity_b

            
            # get ended appointments, which can be used for starting a new appointments.
            a_request_end = A_REQUEST_END
            b_request_end = B_REQUEST_END
            capacity_a = min(capacity_a + a_request_end, MAX_Capacity)
            capacity_b = min(capacity_b + b_request_end, MAX_Capacity)
            value_func += prob * (reward + DISCOUNT * state_value[capacity_a,capacity_b])

    return value_func


#  **Plotting for each iteration(each calculated Policy)(6 in Totall)**

In [27]:
def Plot_Policy(MAX_Capacity = 20,MAX_transfer = 5,A_REQUEST_FIRST = 3,B_REQUEST_FIRST  = 4,
                 A_REQUEST_END = 3,B_REQUEST_END = 2,DISCOUNT = 0.9,CREDIT = 10,COST = 2,UPPER_BOUND = 11):
  
    value = np.zeros((MAX_Capacity + 1, MAX_Capacity + 1)) # We can also have 0 capacity left in each company => 21*21
    policy = np.zeros(value.shape, dtype=np.int) # shape Policy
    actions = np.arange(-MAX_transfer, MAX_transfer + 1)

    iterations = 0
    _, axes = plt.subplots(2, 3, figsize=(40, 20)) # Creat an empty 2*3 figure
    plt.subplots_adjust(wspace=0.1, hspace=0.2) # Adjustments
    axes = axes.flatten() # Convert to a 1D array
    while True:
        fig = sns.heatmap(np.flipud(policy), cmap="viridis", ax=axes[iterations],annot=True) #np.flipud(policy)=> Reverse the order of n array alonge axis=0
        fig.set_ylabel('# Capacity at company A', fontsize=30)
        fig.set_yticks(list(reversed(range(MAX_Capacity + 1))))
        fig.set_xlabel('# Capacity at company B', fontsize=30)
        fig.set_title('policy {}'.format(iterations), fontsize=30)

        # policy evaluation 
        while True:
            old_value = value.copy() # in order to avoid broadcasting
            for i in range(MAX_Capacity + 1):
                for j in range(MAX_Capacity + 1):
                    new_state_value = return_value([i, j], policy[i, j], value,MAX_Capacity, MAX_transfer, A_REQUEST_FIRST,
                                                   B_REQUEST_FIRST, A_REQUEST_END ,B_REQUEST_END ,DISCOUNT ,CREDIT,COST ,UPPER_BOUND)
                    value[i, j] = new_state_value
            max_value_change = abs(old_value - value).max()
            print('max value change {}'.format(max_value_change))
            if max_value_change < 1e-4: # Setting a threshold
                break

        # policy improvement
        policy_stable = True
        for i in range(MAX_Capacity + 1):
            for j in range(MAX_Capacity + 1):
                old_action = policy[i, j]
                action_returns = []
                for action in actions:
                    if (0 <= action <= i) or (-j <= action <= 0):
                        action_returns.append(return_value([i, j], action, value, MAX_Capacity, MAX_transfer, A_REQUEST_FIRST,
                                              B_REQUEST_FIRST, A_REQUEST_END ,B_REQUEST_END ,DISCOUNT ,CREDIT,COST ,UPPER_BOUND))
                    else:
                        action_returns.append(-np.inf)
                new_action = actions[np.argmax(action_returns)]
                policy[i, j] = new_action
                if policy_stable and old_action != new_action:
                    policy_stable = False
        print('policy stable {}'.format(policy_stable))

        if policy_stable:
            fig = sns.heatmap(np.flipud(policy), cmap="viridis", ax=axes[-1],annot=True)
            fig.set_ylabel('# Capacity at company A', fontsize=30)
            fig.set_yticks(list(reversed(range(MAX_Capacity + 1))))
            fig.set_xlabel('# Capacity at company B', fontsize=30)
            fig.set_title('optimal value', fontsize=30)
            break

        iterations += 1

    plt.savefig('figure_Policy_Iteration.png')
    plt.close()



# **Part A**

## Testing for gamma=0.9

In [28]:
Plot_Policy()

max value change 196.62783361783852
max value change 134.98823859766583
max value change 91.41415360228919
max value change 67.17097732555729
max value change 51.29055484635097
max value change 38.49091000659837
max value change 29.406139835126424
max value change 25.7210573245398
max value change 22.381602293031023
max value change 19.40385808254939
max value change 16.77577350573091
max value change 14.47251552455765
max value change 12.464101852186843
max value change 10.719367983418692
max value change 9.20806226246873
max value change 7.9019189666795455
max value change 6.775146571130392
max value change 5.8045764710083745
max value change 4.969618520007145
max value change 4.252112693842776
max value change 3.6361309524054946
max value change 3.107761240497666
max value change 2.654891834022692
max value change 2.26700589940549
max value change 1.9349911763441128
max value change 1.650966802154585
max value change 1.4081276418079938
max value change 1.2006055672075036
max value c

## Testing for gamma=1

In [31]:
Plot_Policy(DISCOUNT = 1)

Streaming output truncated to the last 5000 lines.
max value change 0.0025194275003741495
max value change 0.0025071615527849644
max value change 0.0024949553044280037
max value change 0.0024828084897308145
max value change 0.002470720844939933
max value change 0.002458692000800511
max value change 0.0024467217535857344
max value change 0.0024348097740585217
max value change 0.0024229557948274305
max value change 0.0024111595084832516
max value change 0.0023994206749193836
max value change 0.002387738970355713
max value change 0.0023761141492286697
max value change 0.0023645459241379285
max value change 0.0023530340222350787
max value change 0.002341578176128678
max value change 0.0023301780893234536
max value change 0.002318833494427963
max value change 0.0023075441549735842
max value change 0.0022963097726460546
max value change 0.002285130083691911
max value change 0.0022740048170817317
max value change 0.002262933696329128
max value change 0.00225191650679335
max value change 0.002

# Increasing the Cost from 2 to 6 ( for gamma=0.9)


In [29]:
Plot_Policy(COST=6)

max value change 196.62783361783852
max value change 134.98823859766583
max value change 91.41415360228919
max value change 67.17097732555729
max value change 51.29055484635097
max value change 38.49091000659837
max value change 29.406139835126424
max value change 25.7210573245398
max value change 22.381602293031023
max value change 19.40385808254939
max value change 16.77577350573091
max value change 14.47251552455765
max value change 12.464101852186843
max value change 10.719367983418692
max value change 9.20806226246873
max value change 7.9019189666795455
max value change 6.775146571130392
max value change 5.8045764710083745
max value change 4.969618520007145
max value change 4.252112693842776
max value change 3.6361309524054946
max value change 3.107761240497666
max value change 2.654891834022692
max value change 2.26700589940549
max value change 1.9349911763441128
max value change 1.650966802154585
max value change 1.4081276418079938
max value change 1.2006055672075036
max value c